In [1]:
pip install opencv-python matplotlib transformers timm torch torchvision

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


#Create Transformed Frames

In [ ]:
import cv2
import numpy as np
import os

# Mouse callback function
def select_point(event, x, y, flags, param):
    global src_points
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(temp_frame, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Frame", temp_frame)
        if len(src_points) < 4:
            src_points.append([x, y])

# Load the video for processing
video_path = '/Users/roupenminassian/Downloads/IMG_6362.MP4'
cap = cv2.VideoCapture(video_path)
height, width = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
ret, first_frame = cap.read()

if not ret:
    print("Failed to read the video")
    cap.release()
    exit()

# Display the first frame for point selection
cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", select_point)
temp_frame = first_frame.copy()
src_points = []

print("Select the four points by clicking on the frame. After selecting, press any key to continue.")
cv2.imshow("Frame", first_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Ensure that exactly 4 points were selected
if len(src_points) != 4:
    print("You must select exactly 4 points!")
    cap.release()
    exit()

src_points = np.array(src_points, dtype=np.float32)

dst_width = width
dst_height = int(height * 0.75)
dst_points = np.array([
    [0, dst_height],
    [0, 0],
    [dst_width, 0],
    [dst_width, dst_height]
], dtype=np.float32)

M = cv2.getPerspectiveTransform(src_points, dst_points)

# Folder to save the transformed frames
output_folder = '/Users/roupenminassian/Downloads/Train_6362/'
os.makedirs(output_folder, exist_ok=True)

frame_count = 0

print("Starting frame processing...")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video processing completed!")
        break

    # Transform the frame to bird's-eye view
    warped_frame = cv2.warpPerspective(frame, M, (dst_width, dst_height))

    # Save the transformed frame
    output_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
    cv2.imwrite(output_path, warped_frame)

    print(f"Saved transformed frame {frame_count} to {output_path}")

    frame_count += 1

print("Releasing video capture...")
cap.release()

In [ ]:
import os
import shutil

# Paths to the source folders and destination folder
source_folders = [
    '/Users/roupenminassian/Downloads/TRAIN_0078',
    '/Users/roupenminassian/Downloads/TRAIN_6352',
    '/Users/roupenminassian/Downloads/TRAIN_6353',
    '/Users/roupenminassian/Downloads/TRAIN_6362',
    '/Users/roupenminassian/Downloads/Train Folder'
]
destination_folder = '/Users/roupenminassian/Downloads/Training File'
os.makedirs(destination_folder, exist_ok=True)

# Counter to keep track of file naming
counter = 1

for folder in source_folders:
    for filename in sorted(os.listdir(folder)):
        # Ensure we're processing a file and not a sub-directory
        if os.path.isfile(os.path.join(folder, filename)):
            # Create the new filename based on the counter
            new_filename = f"frame_{counter:04}.jpg"  # This will format the counter as 001, 002, etc.
            # Move and rename the file to the destination folder
            shutil.move(os.path.join(folder, filename), os.path.join(destination_folder, new_filename))
            # Increment the counter
            counter += 1

print("Files merged and renamed successfully!")

In [ ]:
from PIL import Image
import os
import json

def parse_annotation(annotation_path, image_width, image_height):
    objects = {'id': [], 'area': [], 'bbox': [], 'category': []}
    with open(annotation_path, 'r') as f:
        lines = f.readlines()
        for idx, line in enumerate(lines):
            tokens = line.strip().split(' ')
            category = int(tokens[0])
            center_x, center_y, width, height = map(float, tokens[1:])
            center_x *= image_width
            center_y *= image_height
            width *= image_width
            height *= image_height
            
            # Compute bounding box coordinates
            x1 = center_x - (width / 2)
            y1 = center_y - (height / 2)
            x2 = x1 + width
            y2 = y1 + height
            
            # Clip the coordinates to image boundaries
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(image_width, x2)
            y2 = min(image_height, y2)
            
            # Recompute width and height after clipping
            width = x2 - x1
            height = y2 - y1
            
            area = width * height
            objects['id'].append(idx)
            objects['area'].append(area)
            objects['bbox'].append([x1, y1, width, height])
            objects['category'].append(category)
    return objects

def create_dataset_v2(image_folder, annotation_folder):
    dataset = []
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
    image_files.sort()
    
    for i, image_file in enumerate(image_files):
        # Construct the annotation filename from the image filename
        annotation_file = image_file.replace('.jpg', '.txt')
        
        # Ensure the annotation file exists
        if not os.path.exists(os.path.join(annotation_folder, annotation_file)):
            print(f"Annotation for {image_file} not found. Skipping.")
            continue
        
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path)
        image_width, image_height = image.size
        
        annotation_path = os.path.join(annotation_folder, annotation_file)
        objects = parse_annotation(annotation_path, image_width, image_height)
        
        dataset_entry = {
            'image_id': i,
            'file_name': image_file,
            'width': image_width,
            'height': image_height,
            'objects': objects
        }
        dataset.append(dataset_entry)
    return dataset

# Paths to image and annotation folders
image_folder = '/Users/roupenminassian/Downloads/Training File/'
annotation_folder = '/Users/roupenminassian/Downloads/Annotations File/'

dataset_v2 = create_dataset_v2(image_folder, annotation_folder)

# Show first dataset entry
dataset_v2[0] if dataset_v2 else "No valid dataset entries found."

In [ ]:
import json

def save_to_jsonl_v2(dataset, output_file_path):
    """
    Save the dataset to a JSONL (JSON Lines) file in the new format.
    
    Parameters:
    - dataset: List of dataset entries
    - output_file_path: Path to the output JSONL file
    """
    with open(output_file_path, 'w') as f:
        for entry in dataset:
            # Prepare the entry in the desired output format
            output_entry = {
                'file_name': entry['file_name'],  # Use the actual file name
                'image_id': entry['image_id'],
                'width': entry['width'],
                'height': entry['height'],
                'objects': {
                    'id': entry['objects']['id'],
                    'area': entry['objects']['area'],
                    'bbox': entry['objects']['bbox'],
                    'category': entry['objects']['category']
                }
            }
            # Write each entry as a JSON object on a new line
            f.write(json.dumps(output_entry) + '\n')

# Path to the JSONL file
output_file_path = '/Users/roupenminassian/Downloads/Training/metadata.jsonl'
save_to_jsonl_v2(dataset_v2, output_file_path)

# Verify the first few lines from the generated JSONL file
with open(output_file_path, 'r') as f:
    sample_lines_v2 = [next(f) for _ in range(min(20, len(dataset_v2)))]

sample_lines_v2

In [ ]:
import json
import shutil
import os

def copy_annotated_images(jsonl_path, src_folder, dest_folder):
    # Ensure the destination folder exists
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    
    # Open the jsonl file and iterate through each line
    with open(jsonl_path, 'r') as f:
        for line in f:
            data = json.loads(line)
            file_name = data['file_name']
            src_path = os.path.join(src_folder, file_name)
            dest_path = os.path.join(dest_folder, file_name)
            
            # Check if the file exists in the source folder
            if os.path.exists(src_path):
                # Copy the file to the destination folder
                shutil.copy2(src_path, dest_path)

# Path to the jsonl file
jsonl_path = '/Users/roupenminassian/Downloads/Training/metadata.jsonl'
# Source folder containing the images
src_folder = '/Users/roupenminassian/Downloads/Training/Training File/'
# Destination folder where the images will be copied
dest_folder = '/Users/roupenminassian/Downloads/Training/Train/'

# Call the function
copy_annotated_images(jsonl_path, src_folder, dest_folder)

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

folderPath = "/Users/roupenminassian/Downloads/Training"

dataset = load_dataset("imagefolder", data_dir=folderPath)
dataset.push_to_hub("roupenminassian/vehicle-dataset")